In [1]:
#using sklearn random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from talib import abstract
from ccxt import binance
import pandas as pd
import numpy as np
import os

In [2]:
os.remove("../data/BTC_USDT_30m.csv")

In [3]:
binance = binance()
symbol = 'BTC/USDT'
timeframe = '30m'
file_name = f"../data/{symbol.replace('/', '_')}_{timeframe}.csv"
start = binance.parse8601('2019-10-01T00:00:00Z')
end = binance.parse8601('2022-12-30T00:00:00Z')
cnt_time = start
data = []
while cnt_time < end:
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, cnt_time)
    data += ohlcv
    cnt_time = ohlcv[-1][0] + 1800000 # 1h in ms    
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='ms')
df.to_csv(file_name, index=False)

In [4]:
df

,time,open,high,low,close,volume
0,2019-10-01 00:00:00,8289.97,8310.08,8206.57,8220.39,1407.679668
1,2019-10-01 00:30:00,8220.78,8222.13,8188.00,8199.66,462.495135
2,2019-10-01 01:00:00,8199.21,8229.99,8173.05,8203.58,646.261799
3,2019-10-01 01:30:00,8204.99,8295.00,8193.57,8269.83,738.759621
4,2019-10-01 02:00:00,8269.83,8428.00,8265.14,8388.60,2763.540693
...,...,...,...,...,...,...
56878,2022-12-30 15:00:00,16414.18,16537.33,16400.40,16503.07,8734.614660
56879,2022-12-30 15:30:00,16503.76,16580.70,16502.24,16555.26,7322.280260
56880,2022-12-30 16:00:00,16555.26,16577.00,16517.31,16520.66,4773.922960
56881,2022-12-30 16:30:00,16520.14,16551.64,16515.45,16545.86,3067.765670


In [5]:
df['MFI'] = abstract.MFI(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['ADX'] = abstract.ADX(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['ATR'] = abstract.ATR(df, timeperiod=14)
df['OBV'] = abstract.OBV(df, timeperiod=14)
df['EMA'] = abstract.EMA(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['AD'] = abstract.AD(df, timeperiod=14)
df['ADOSC'] = abstract.ADOSC(df, timeperiod=14)
df['ADXR'] = abstract.ADXR(df, timeperiod=14)
df['APO'] = abstract.APO(df, timeperiod=14)
df['AROONOSC'] = abstract.AROONOSC(df, timeperiod=14)
df['BOP'] = abstract.BOP(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['CMO'] = abstract.CMO(df, timeperiod=14)
df['DX'] = abstract.DX(df, timeperiod=14)
df['MOM'] = abstract.MOM(df, timeperiod=14)
df['PPO'] = abstract.PPO(df, timeperiod=14)
df['ROC'] = abstract.ROC(df, timeperiod=14)
df['ROCP'] = abstract.ROCP(df, timeperiod=14)
df['ROCR'] = abstract.ROCR(df, timeperiod=14)
df['ROCR100'] = abstract.ROCR100(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['TRIX'] = abstract.TRIX(df, timeperiod=14)
df['ULTOSC'] = abstract.ULTOSC(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['WMA'] = abstract.WMA(df, timeperiod=14)
df['HT_TRENDLINE'] = abstract.HT_TRENDLINE(df, timeperiod=14)
df['TRANGE'] = abstract.TRANGE(df, timeperiod=14)
df['CLOSE_percent'] = df['close'].pct_change()

In [6]:
df['UP'] = df['CLOSE_percent'].apply(lambda x: 1 if x > 0 else 0)
df['DOWN'] = df['CLOSE_percent'].apply(lambda x: 1 if x < 0 else 0)
df['UP'] = df['UP'].shift(-1) #shift UP DOWN 一個單位，因為我們要預測的是下一個時間點的漲跌
df['DOWN'] = df['DOWN'].shift(-1)

df = df.dropna()

In [7]:
#正規化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']] = scaler.fit_transform(df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']])
df[['open','high','low','close','volume']] = scaler.fit_transform(df[['open','high','low','close','volume']])

C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [8]:
X = df.drop(['close','open','high','low','volume','UP','DOWN','time'], axis=1)
#y -> UP and DOWN
y = df['UP'] #UP

In [9]:
X = X.values
y = y.values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [11]:
X_train

array([[ 6.01001278e-01,  5.93489767e-01,  2.84192255e-01, ...,
         1.67277153e-01,  1.74464571e-02,  5.79169214e-04],
       [ 5.47043348e-01,  6.46787545e-01,  2.33580136e-01, ...,
         2.20159440e-01,  1.28948000e-02,  1.18040794e-03],
       [ 6.25743785e-01,  5.99658899e-01,  1.19705525e-01, ...,
         7.68448610e-01,  5.27434686e-02, -2.56112610e-03],
       ...,
       [ 5.92007663e-01,  5.14257386e-01,  1.20999904e-01, ...,
         6.40179773e-02,  4.41344414e-03,  5.91144077e-04],
       [ 6.47860229e-01,  5.41801156e-01,  2.71412613e-01, ...,
         7.22531877e-02,  2.80525443e-03, -7.59492334e-04],
       [ 4.79123737e-01,  3.95881391e-01,  1.37330375e-01, ...,
         6.97476097e-01,  1.37978694e-01, -1.95965477e-02]])

In [12]:
#random forest classifier with n_estimators=10 (default)
model = RandomForestClassifier(
    n_estimators=10000,
    criterion='gini',
    max_features=10,
    max_depth=8)



In [13]:
#train the model using the training sets y_pred=clf.predict(X_test)
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=8, max_features=10, n_estimators=10000)

In [14]:
pred = model.predict(X_test)

print(accuracy_score(y_test, pred))

0.5517423442449841


In [15]:
pred

array([1., 0., 0., ..., 1., 0., 1.])

In [16]:
X_train[0]

array([6.01001278e-01, 5.93489767e-01, 2.84192255e-01, 4.50198774e-01,
       9.65438295e-02, 2.67405684e-01, 1.70463761e-01, 4.70344437e-01,
       4.43279015e-01, 5.92664108e-01, 2.94367031e-01, 6.19810191e-01,
       7.50000000e-01, 5.35951662e-01, 5.93489767e-01, 1.19786628e-01,
       5.92039504e-01, 7.45479661e-01, 5.63615588e-01, 5.63615588e-01,
       5.63615588e-01, 5.63615588e-01, 7.32968140e-01, 4.61886854e-01,
       1.72318548e-01, 1.67277153e-01, 1.74464571e-02, 5.79169214e-04])

In [17]:
test_ans = model.predict(X_train)
print(accuracy_score(y_train, test_ans))

0.6371136288637114


In [18]:
#confusion matrix
confusion_matrix(y_test, pred)


array([[2921, 2688],
       [2406, 3349]], dtype=int64)

In [19]:
#classification report  
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.55      0.52      0.53      5609
         1.0       0.55      0.58      0.57      5755

    accuracy                           0.55     11364
   macro avg       0.55      0.55      0.55     11364
weighted avg       0.55      0.55      0.55     11364



In [20]:
#ROC curve
fpr, tpr, thresholds = roc_curve(y_test, pred)
auc = roc_auc_score(y_test, pred)
print('AUC: %.2f' % auc)

AUC: 0.55
